In [1]:
%pip install -q langchain_openai langchain pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

import os
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['GEMINI_API_KEY'] = userdata.get('gemini_api_key')

In [3]:
from langchain_openai import ChatOpenAI

In [18]:
model = ChatOpenAI(model='gpt-4o' ,openai_api_key=openai_api_key) #gpt-4-0125-preview  gpt-3.5-turbo-0125


In [ ]:
# pdf_file =

In [5]:
# prompt: write a code to give a zip file path and extract it

import zipfile
import os

def extract_zip(zip_file_path, extract_to_path):
    """Extracts a zip file to a specified directory.

    Args:
        zip_file_path: The path to the zip file.
        extract_to_path: The directory to extract the zip file to.
    """
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to_path)
        print(f"Successfully extracted '{zip_file_path}' to '{extract_to_path}'")
    except FileNotFoundError:
        print(f"Error: Zip file not found at '{zip_file_path}'")
    except zipfile.BadZipFile:
        print(f"Error: Invalid zip file at '{zip_file_path}'")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


for zip_file_path in ["/content/PO.zip", "/content/Invoice.zip"]:
    extract_to_path = "."  # Replace with the desired extraction directory
    extract_zip(zip_file_path, extract_to_path)


Successfully extracted '/content/PO.zip' to '.'
Successfully extracted '/content/Invoice.zip' to '.'


In [6]:
import pdfplumber

In [7]:
def read_pdf(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [23]:
path_2_pdf = '/content/Invoice/8745-1691067610.pdf'
path_2_pdf = '/content/PO/4681-1689166172.pdf'

text = read_pdf(path_2_pdf)

In [9]:
text

'Invoice\nNo: 1622487\nDate: 09 June 2023\nAccounts Payable\nBuzz Wireless Network\nOffice 2\nThe Courtyard\nBawtry\nDoncaster\nDN10 6JG\nKNW No. 01302 710 819\nSupplier No. Kieran Feenix\nPurchase order\nRE: AMOUNT\nLantra-Landscape Tools Multi-tool\n26th - 27th June 2023 938.00\nAccreditation 4 Candidates\n464.00\nEquipment Hire - 4 Candidates\n32.00\nSite Hire\n190.00\nNet Amount\n1,624.00\nVAT @ 20% 324.80\nTotal Due £ 1,948.80\nPayment due immediately\nPlease note bank details\nNational Westminster Bank PLC\n2, Howe Walk, Burnley, BB11 1NZ\nA/c No: 17887623\nA/c Name: Kielder Newport West Ltd\nSort Code: 01-01-35\nPlease make all cheques payable to Kielder Newport West Limited\nIf you have any queries regarding this invoice please contact KNW on (01677) 424633\nVAT Reg No: 693 0290 30'

In [19]:
from pydantic import BaseModel, Field
from typing import Optional

class PurchaseOrder(BaseModel):
    """Schema for extracting purchase order details from text."""

    company_name: str = Field(
        description="The name of the company related to the purchase order. Ensure the name does not include 'BUZZ WIRELESS NETWORKS LTD.'"
    )

    purchase_order: str = Field(...,
        description="The unique identifier for the purchase order. This could also be referred to as 'purchase No' or 'order No'."
    )

    order_no: str = Field(
        description="The order number for the transaction. It is used interchangeably with 'purchase order' or 'purchase No'."
    )

    quote_ref: Optional[str] = Field(
        default=None, description="The reference number for the quote associated with the order. It may or may not be present."
    )

    net_amount: int = Field(
        description="The net amount for the purchase, excluding any additional charges or taxes."
    )

    total_due: int = Field(
        description="The total amount due for the purchase, which includes any applicable taxes or charges."
    )


In [20]:
model_with_structure = model.with_structured_output(PurchaseOrder)

In [21]:
model_with_structure.invoke('hey, how are you?')

PurchaseOrder(company_name='Advanced Tech Innovations', purchase_order='PO123456', order_no='ORD987654', quote_ref='QR567890', net_amount=1500, total_due=1635)

In [24]:
model_with_structure.invoke(text)

PurchaseOrder(company_name='Clarke Telecom Ltd', purchase_order='P271070', order_no='P271070', quote_ref=None, net_amount=4235, total_due=4235)

In [25]:
from pydantic import BaseModel, Field
from typing import Optional

class Invoice(BaseModel):
    """Schema for extracting invoice details from text."""

    company_name: str = Field(
        description="The name of the company on the invoice. 'BUZZ WIRELESS NETWORKS LTD.' should not appear."
    )

    purchase_order: str = Field(
        description="The identifier for the purchase order associated with the invoice. Could be referred to as 'purchase No' or 'order No'."
    )

    order_no: str = Field(
        description="Order number for the transaction, used interchangeably with 'purchase order' and 'purchase No'."
    )

    invoice_no: str = Field(
        description="The unique invoice number for the transaction."
    )

    date: str = Field(
        description="The date when the invoice was issued. Usually formatted as 'DD Month YYYY'."
    )

    sort_code: str = Field(
        description="The sort code for the bank account associated with the invoice."
    )

    net_amount: int = Field(
        description="The net amount before tax and additional charges."
    )

    vat_20: int = Field(
        description="The VAT (Value Added Tax) at 20% applicable to the net amount."
    )

    total_due: int = Field(
        description="The total amount due, which includes the VAT and any additional charges."
    )

    account_number: str = Field(
        description="The bank account number for payment processing."
    )

    ac_name: str = Field(
        description="The name associated with the account where the payment should be made."
    )


In [26]:
invoice_structure = model.with_structured_output(Invoice)

In [30]:
pdf_file = '/content/Invoice/8603-1691675777.pdf'

text = read_pdf(pdf_file)

In [31]:
invoice_structure.invoke(text)

Invoice(company_name='Fields And Towers Limited', purchase_order='EM01-2301', order_no='EM01-2301', invoice_no='0000087513', date='18/04/2023', sort_code='30-95-05', net_amount=500, vat_20=100, total_due=600, account_number='02622564', ac_name='Fields And Towers (Communications) Ltd')

In [40]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()


completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the Invoice information."},
        {"role":"user", "content": f"here is the invoice's text. work on it:\n\n{text}"}
    ],
    response_format=Invoice,
)

invoice_information = completion.choices[0].message.parsed

In [41]:
text

'##CFG SalesInvoices ## ##VAR1 BUZ001## ##VAR2 0000087513## ##COMPANY Buzz Wireless\nNetworks## ##VAR3 EM01-2301## Fields And Towers Limited\nBraidhurst House\n2 Finch Way\nStrathclyde Business Park\nBellshill\nML4 3PE\nTel : 01698 842426\n##NAME ## ##EMAIL invoices@buzzwn.co.uk## ##FAX ##\nWeb: www.wirelessinfrastructure.co.uk\nEmail: accounts_receivable@wirelessinfrastructure.co.uk\nInvoice COPY\nInvoice No: 0000087513\nBuzz Wireless Networks\nInvoice Date: 18/04/2023\nOffice 2 The Courtyard\nBawtry Account No: BUZ001\nDoncaster\nYour Order No: EM01-2301\nDN10 6JG\nGreat Britain\nAll Prices In: Pound Sterling\nSite Name: Pather Farm\nOur Ref: 356738\nYour Ref: Northern England, 03/04/2023\nStatus: PPM - Annual Inspection Date To: 03/04/2023 Requestor:Evie Pritchard\nDetails Qty Unit Price Net VAT %\nAccess Invoice\nIncurred costs to the above site 1 500.00 500.00 20.00\nAny queries should be raised within 21 days of the invoice date; otherwise the invoiced amount will be deemed\nto b

In [42]:
invoice_information

Invoice(company_name='Fields And Towers Limited', purchase_order='EM01-2301', order_no='EM01-2301', invoice_no='0000087513', date='18/04/2023', sort_code='30-95-05', net_amount=500, vat_20=100, total_due=600, account_number='02622564', ac_name='Fields And Towers (Communications) Ltd')

In [45]:
from google import genai
from pydantic import BaseModel



g_client = genai.Client(api_key=userdata.get('gemini_api_key'))
response = g_client.models.generate_content(
    model='gemini-2.0-flash',
    contents=f"Extract the Invoice information.invoice's text is:\n\n{text}",
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Invoice],
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
# response.parsed[0]

[
  {
    "company_name": "Buzz Wireless Networks",
    "purchase_order": "EM01-2301",
    "order_no": "EM01-2301",
    "invoice_no": "0000087513",
    "date": "18/04/2023",
    "sort_code": "30-95-05",
    "net_amount": 500,
    "vat_20": 100,
    "total_due": 600,
    "account_number": "02622564",
    "ac_name": "Fields And Towers (Communications) Ltd"
  }
]


In [ ]:
import os

def list_files_with_path(folder_path):
    """Lists all files in the given folder and returns their complete paths.

    Args:
        folder_path: The path to the folder.

    Returns:
        A list of strings, where each string is the complete path of a file
        in the folder. Returns an empty list if the folder does not exist
        or if there are no files in the folder.
    """
    file_paths = []
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            if os.path.isfile(file_path):
                file_paths.append(file_path)
    return file_paths

# Example usage (assuming you have a folder named "my_folder"):
folder_path = "/content/Invoice"  # Replace with the actual folder path
file_paths = list_files_with_path(folder_path)

if file_paths:
    print("Files in the folder:")
    for file_path in file_paths:
        print(file_path)
else:
    print(f"The folder '{folder_path}' does not exist or contains no files.")


In [47]:
import json

results = {}
g_client = genai.Client(api_key=userdata.get('gemini_api_key'))

for file_path in file_paths[:5]:
    try:
        text = read_pdf(file_path)
        response = g_client.models.generate_content(
            model='gemini-2.0-flash',
            contents=f"Extract the Invoice information from the following text:\n{text}",
            config={
                'response_mime_type': 'application/json',
                'response_schema': list[Invoice],
            },
        )
        # Attempt to parse the JSON response
        try:
            parsed_response = json.loads(response.text)
            results[file_path] = parsed_response
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for {file_path}: {e}")
            results[file_path] = {"error": f"JSON decoding error: {e}", "raw_response": response.text}
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        results[file_path] = {"error": str(e)}

results


{'/content/Invoice/8946-1696581896.pdf': [{'company_name': '2JCT RIGGING LTD',
   'purchase_order': 'N/A',
   'order_no': 'N/A',
   'invoice_no': '9',
   'date': '01/09/2023',
   'sort_code': '53 50 39',
   'net_amount': 5130,
   'vat_20': 0,
   'total_due': 5130,
   'account_number': '55433022',
   'ac_name': 'James Howick'}],
 '/content/Invoice/8931-1696577535.pdf': [{'company_name': 'Dawsongroup Vans Ltd',
   'purchase_order': '',
   'order_no': '',
   'invoice_no': '3118647',
   'date': '23/08/2023',
   'sort_code': '',
   'net_amount': 22,
   'vat_20': 4,
   'total_due': 27,
   'account_number': '52515',
   'ac_name': ''}],
 '/content/Invoice/8815-1691335008.pdf': [{'company_name': 'MCS Test Equipment Ltd',
   'purchase_order': 'PO KC01-1081',
   'order_no': '1125',
   'invoice_no': '20381',
   'date': '13 Jul 2023',
   'sort_code': '40-30-18',
   'net_amount': 627,
   'vat_20': 125,
   'total_due': 752,
   'account_number': '31284363',
   'ac_name': 'MCS Test Equipment Ltd'}],
 '

In [48]:
import pandas as pd

def dict_of_dict_to_pandas(input_dict):
  """Converts a dictionary of dictionaries to a pandas DataFrame.

  Args:
    input_dict: A dictionary where keys are file paths and values are lists
                of dictionaries, each representing a record.

  Returns:
    A pandas DataFrame containing the data.
  """

  all_records = []
  for file_path, records in input_dict.items():
    for record in records:
      record['file_path'] = file_path  # Add file path as a column
      all_records.append(record)

  return pd.DataFrame(all_records)

df = dict_of_dict_to_pandas(results) # use the results dictionary from the code
df


,company_name,purchase_order,order_no,invoice_no,date,sort_code,net_amount,vat_20,total_due,account_number,ac_name,file_path
0,2JCT RIGGING LTD,N/A,N/A,9,01/09/2023,53 50 39,5130,0,5130,55433022,James Howick,/content/Invoice/8946-1696581896.pdf
1,Dawsongroup Vans Ltd,,,3118647,23/08/2023,,22,4,27,52515,,/content/Invoice/8931-1696577535.pdf
2,MCS Test Equipment Ltd,PO KC01-1081,1125,20381,13 Jul 2023,40-30-18,627,125,752,31284363,MCS Test Equipment Ltd,/content/Invoice/8815-1691335008.pdf
3,BuzzWirelessNetworks,N/A,N/A,INV-9707,26Jun2023,403438,360,72,432,31487299,ConeyExpress,/content/Invoice/8748-1691068262.pdf
4,Smith Craven,N/A,N/A,82587,30/06/2023,20-55-58,638,127,765,00272477,Barclays Bank,/content/Invoice/8875-1692964285.pdf
